# Variants in DNAJC13 and their Association with Parkinson's Disease Across Different Ancestral Backgrounds

* **Project**: DNAJC13 Gene Assesment
* **Last updated**: September 2024
* **Version**: Python 3.9
* **Data**: AMP-PD Release 3

# Summary
This notebook is focused on analysing association between DNAJC13 variants in Ashkenazi Jewish (AJ) ancestry and Parkinson's disease.

# Imports

In [1]:
# Use the os package to interact with the environment
import os
import sys

# Bring in Pandas for Dataframe functionality
import pandas as pd
from functools import reduce

# Bring some visualization functionality 
import seaborn as sns

# numpy for basics
import numpy as np

# Use StringIO for working with file contents
from io import StringIO

# Enable IPython to display matplotlib graphs
import matplotlib.pyplot as plt
%matplotlib inline

# Enable interaction with the FireCloud API
from firecloud import api as fapi

# Import the iPython HTML rendering for displaying links to Google Cloud Console
from IPython.core.display import display, HTML

# Import urllib modules for building URLs to Google Cloud Console
import urllib.parse

# BigQuery for querying data
from google.cloud import bigquery

/tmp/ipykernel_6380/3943855956.py:26: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
# Utility routine for printing a shell command before executing it
def shell_do(command):
    print(f'Executing: {command}', file=sys.stderr)
    !$command
    
def shell_return(command):
    print(f'Executing: {command}', file=sys.stderr)
    output = !$command
    return '\n'.join(output)

# Utility routine for printing a query before executing it
def bq_query(query):
    print(f'Executing: {query}', file=sys.stderr)
    return pd.read_gbq(query, project_id=BILLING_PROJECT_ID, dialect='standard')

# Utility routine for display a message and a link
def display_html_link(description, link_text, url):
    html = f'''
    <p>
    </p>
    <p>
    {description}
    <a target=_blank href="{url}">{link_text}</a>.
    </p>
    '''

    display(HTML(html))

# Utility routines for reading files from Google Cloud Storage
def gcs_read_file(path):
    """Return the contents of a file in GCS"""
    contents = !gsutil -u {BILLING_PROJECT_ID} cat {path}
    return '\n'.join(contents)
    
def gcs_read_csv(path, sep=None):
    """Return a DataFrame from the contents of a delimited file in GCS"""
    return pd.read_csv(StringIO(gcs_read_file(path)), sep=sep, engine='python')

# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_gcs(description, link_text, gcs_path):
    url = '{}?{}'.format(
        os.path.join('https://console.cloud.google.com/storage/browser',
                     gcs_path.replace("gs://","")),
        urllib.parse.urlencode({'userProject': BILLING_PROJECT_ID}))

    display_html_link(description, link_text, url)

In [ ]:
# Set up billing project and data path variables
BILLING_PROJECT_ID = os.environ['GOOGLE_PROJECT']
WORKSPACE_NAMESPACE = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE_NAME = os.environ['WORKSPACE_NAME']
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']

WORKSPACE_ATTRIBUTES = fapi.get_workspace(WORKSPACE_NAMESPACE, WORKSPACE_NAME).json().get('workspace',{}).get('attributes',{})

## AMP-PD v2.5
AMP_CLINICAL_RELEASE_PATH = f'{AMP_RELEASE_PATH}/clinical'

AMP_WGS_RELEASE_PLINK_PATH = os.path.join(AMP_WGS_RELEASE_PATH, 'plink')
AMP_WGS_RELEASE_GATK_PATH = os.path.join(AMP_WGS_RELEASE_PATH, 'gatk')

## Print the information to check we are in the proper release and billing 
## This will be different for you, the user, depending on the billing project your workspace is on
print('Billing and Workspace')
print(f'Workspace Name @ `WORKSPACE_NAME`: {WORKSPACE_NAME}')
print(f'Billing Project @ `BILLING_PROJECT_ID`: {BILLING_PROJECT_ID}')
print(f'Workspace Bucket, where you can upload and download data @ `WORKSPACE_BUCKET`: {WORKSPACE_BUCKET}')
print('')

print('AMP-PD v2.5')
print(f'Path to AMP-PD v2.5 Clinical Data @ `AMP_CLINICAL_RELEASE_PATH`: {AMP_CLINICAL_RELEASE_PATH}')
print(f'Path to AMP-PD v2.5 WGS Data @ `AMP_WGS_RELEASE_PLINK_PATH`: {AMP_WGS_RELEASE_PLINK_PATH}')
print('')

## GP2 v5.0
## Explicitly define release v5.0 path 
GP2_CLINICAL_RELEASE_PATH = f'{GP2_RELEASE_PATH}/clinical_data'
GP2_META_RELEASE_PATH = f'{GP2_RELEASE_PATH}/meta_data'
GP2_SUMSTAT_RELEASE_PATH = f'{GP2_RELEASE_PATH}/summary_statistics'

GP2_RAW_GENO_PATH = f'{GP2_RELEASE_PATH}/raw_genotypes'
GP2_IMPUTED_GENO_PATH = f'{GP2_RELEASE_PATH}/imputed_genotypes'
print('GP2 v6.0')
print(f'Path to GP2 v6.0 Clinical Data @ `GP2_CLINICAL_RELEASE_PATH`: {GP2_CLINICAL_RELEASE_PATH}')
print(f'Path to GP2 v6.0 Metadata @ `GP2_META_RELEASE_PATH`: {GP2_META_RELEASE_PATH}')
print(f'Path to GP2 v6.0 Raw Genotype Data @ `GP2_RAW_GENO_PATH`: {GP2_RAW_GENO_PATH}')
print(f'Path to GP2 v6.0 Imputed Genotype Data @ `GP2_IMPUTED_GENO_PATH`: {GP2_IMPUTED_GENO_PATH}')

# Make working directory

In [ ]:
ancestry = "AJ"
WORK_DIR = f'2024_BURDEN_AMP_{ancestry}_GLM'
! mkdir {WORK_DIR}

AMP_DATA_DIR = f'AMP_DATA_DIR'
! mkdir {AMP_DATA_DIR}


# Install Packages

## PLINK

In [ ]:
%%bash

mkdir -p ~/tools
cd ~/tools

if test -e /home/jupyter/tools/plink; then
echo "Plink1.9 is already installed in /home/jupyter/tools/"

else
echo -e "Downloading plink \n    -------"
wget -N http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20190304.zip 
unzip -o plink_linux_x86_64_20190304.zip
echo -e "\n plink downloaded and unzipped in /home/jupyter/tools \n "

fi

In [ ]:
%%bash

mkdir -p ~/tools
cd ~/tools

if test -e /home/jupyter/tools/plink2; then
echo "Plink2 is already installed in /home/jupyter/tools/"

else
echo -e "Downloading plink2 \n    -------"
wget -N https://s3.amazonaws.com/plink2-assets/alpha3/plink2_linux_avx2_20220603.zip
unzip -o plink2_linux_avx2_20220603.zip
echo -e "\n plink2 downloaded and unzipped in /home/jupyter/tools \n "

fi

## ANNOVAR

In [ ]:
%%bash

# Install ANNOVAR:
# https://www.openbioinformatics.org/annovar/annovar_download_form.php

if test -e /home/jupyter/tools/annovar; then

echo "annovar is already installed in /home/jupyter/tools/"
else
echo "annovar is not installed"
cd /home/jupyter/tools/

wget http://www.openbioinformatics.org/annovar/download/0wgxR2rIVP/annovar.latest.tar.gz

tar xvfz annovar.latest.tar.gz

fi

In [ ]:
%%bash

# Install ANNOVAR: Download resources for annotation

cd /home/jupyter/tools/annovar/

perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar refGene humandb/
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar clinvar_20140902 humandb/
#perl annotate_variation.pl -buildver hg38 -downdb cytoBand humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ensGene humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar exac03 humandb/ 
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar avsnp147 humandb/ 
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar dbnsfp30a humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar gnomad211_genome humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ljb26_all humandb/

## RVTests

In [ ]:
%%bash

#Install RVTESTS: Option 1 (~15min)
if test -e /home/jupyter/tools/rvtests; then

echo "rvtests is already installed"
else
echo "rvtests is not installed"

mkdir /home/jupyter/tools/rvtests
cd /home/jupyter/tools/rvtests

wget https://github.com/zhanxw/rvtests/releases/download/v2.1.0/rvtests_linux64.tar.gz 

tar -zxvf rvtests_linux64.tar.gz
fi

In [ ]:
# chmod to make sure you have permission to run the program
! chmod u+x /home/jupyter/tools/plink
! chmod u+x /home/jupyter/tools/plink2
! chmod 777 /home/jupyter/tools/rvtests/executable/rvtest

# Copy Over Files 

In [ ]:
## chr 3
# Commented out because it is already downloaded
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {AMP_WGS_RELEASE_PLINK_PATH}/pfiles/chr3.pgen {AMP_DATA_DIR}/chr3.pgen.new')

In [ ]:
## clinical data 

## demographics
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {AMP_CLINICAL_RELEASE_PATH}/Demographics.csv {WORK_DIR}')

## enrollment
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {AMP_CLINICAL_RELEASE_PATH}/Enrollment.csv {WORK_DIR}')

## duplicates
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {AMP_RELEASE_PATH}/amp_pd_participant_wgs_duplicates.csv {WORK_DIR}')

## related file 
#shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {GP2_META_RELEASE_PATH}/related_samples/release6_{ancestry}.related {WORK_DIR}')

In [ ]:
# refFlat
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {WORKSPACE_BUCKET}/notebooks/misc/refFlat_HG38_all_chr.txt {WORK_DIR}')

In [ ]:
# Predicted ancestries uploaded by Mary
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {WORKSPACE_BUCKET}/AMPPD_v25_GenoTools_Predictions.txt {WORK_DIR}')


In [ ]:
# Additional files uploaded by Mary
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {WORKSPACE_BUCKET}/AMP_v3/AMPPD_v3_COV_wPHENOS_wGENOTOOLS.csv {WORK_DIR}')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {WORKSPACE_BUCKET}/AMP_v3/PCA.FILTERED.AMP_PD_{ancestry}.PD.eigenvec {WORK_DIR}')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {WORKSPACE_BUCKET}/AMP_v3/toRemove_1stand2ndDegree_Relateds_{ancestry}.txt {WORK_DIR}')

In [ ]:
## Check files
! ls {WORK_DIR}

# Covariate File

In [ ]:
# Let's load the disease status
enrollment = pd.read_csv(f'{WORK_DIR}/Enrollment.csv')
print(enrollment.shape)
enrollment.head()

In [ ]:
# Let's load the demographic data
demographics = pd.read_csv(f'{WORK_DIR}/Demographics.csv')
print(demographics.shape)
demographics.head()

In [ ]:
# Let's load the predicted ancestry
predictedAncestry= pd.read_csv(f'{WORK_DIR}/AMPPD_v25_GenoTools_Predictions.txt', sep='\s+' )
print(predictedAncestry.shape)
predictedAncestry.head()

In [ ]:
#Get just the needed info from Enrollment
enrollment = enrollment[['participant_id', 'study_arm']]
enrollment.rename(columns = {'participant_id': 'IID',
                      'study_arm':'phenotype'}, inplace = True)
print(enrollment.shape)
enrollment.head()

In [ ]:
#Get just the needed info from Demographics
demographics = demographics[['participant_id', 'sex', 'age_at_baseline']]
demographics.rename(columns = {'participant_id': 'IID',
                      'sex':'SEX',
                      'age_at_baseline':'AGE'}, inplace = True)
print(demographics.shape)
demographics.head()

In [ ]:
# Merge Enrollment with Demographics
merged_key = pd.merge(enrollment, demographics, on='IID', how='left')
print(merged_key.shape)
merged_key.head()

In [ ]:
# Merge Enrollment/Demographics with Ancestry
merged_key2 = pd.merge(merged_key, predictedAncestry, on='IID', how='left')
print(merged_key2.shape)
merged_key2.head()

In [ ]:
merged_key2['label'].value_counts(dropna=False)

In [ ]:
merged_key2['phenotype'].value_counts(dropna=False)

In [ ]:
merged_key2[merged_key2['label']==ancestry]

In [ ]:
## Subset to keep ancestry of interest 
ancestry_key = merged_key2[merged_key2['label']==ancestry].copy()
ancestry_key.reset_index(drop=True)

In [ ]:
# Convert phenotype to binary (0/1)
    # PD = 1; control = 0
pheno_mapping = {"PD": 2, "Healthy Control": 1}
ancestry_key['PHENO'] = ancestry_key['phenotype'].map(pheno_mapping).astype('Int64')

In [ ]:
# Check value counts of pheno
ancestry_key['PHENO'].value_counts(dropna=False)

In [ ]:
# Convert sex to binary (0/1)
    # Female = 1; male = 0
sex_mapping = {'Female': 2, 'Male': 1}
ancestry_key['SEX'] = ancestry_key['SEX'].map(sex_mapping).astype('Int64')

In [ ]:
# Check value counts of SEX
ancestry_key['SEX'].value_counts(dropna=False)

In [ ]:
## Clean up and keep columns we need 
final_df = ancestry_key[['FID','IID', 'SEX', 'PHENO', 'AGE']].copy()
final_df

In [ ]:
# remove duplicate values in IID column
final_df = final_df.drop_duplicates(subset=['IID'], keep='first')

In [ ]:
final_df.groupby(['PHENO'])['SEX'].value_counts()

In [ ]:
final_df.dropna(inplace=True)

In [ ]:
final_df.groupby(['PHENO'])['SEX'].value_counts()

In [ ]:
print(final_df.shape)
final_df.head()

In [ ]:
# Load information about related individuals 
column_names = ['FID','IID']
related_df = pd.read_csv(f'{WORK_DIR}/toRemove_1stand2ndDegree_Relateds_{ancestry}.txt', sep='\t', header = None, names=column_names)
print(related_df.shape)
related_df

In [ ]:
# Make a list of just one set of related people
related_list = list(related_df['IID'])

# Check value counts of related and remove only one related individual
final_df = final_df[~final_df["IID"].isin(related_list)]

# Check size
print(final_df.shape)
final_df

In [ ]:
## Make file of sample IDs to keep 
samples_toKeep = final_df[['FID','IID']].copy()
samples_toKeep.to_csv(f'{WORK_DIR}/{ancestry}.samplestoKeep', sep = '\t', index=False, header=None)

In [ ]:
! head {WORK_DIR}/{ancestry}.samplestoKeep

In [ ]:
#final_df['FID'] = 0
final_df['FATID'] = 0
final_df['MATID'] = 0

In [ ]:
final_df2=final_df[['FID','IID','MATID','FATID','SEX','PHENO', 'AGE']].copy()

In [ ]:
final_df2

In [ ]:
## Load PCs
pcs = pd.read_csv(f'{WORK_DIR}/PCA.FILTERED.AMP_PD_{ancestry}.PD.eigenvec', sep='\t')
print(pcs.shape)
pcs

In [ ]:
# Merge Cov and PCs
final_df3 = pd.merge(final_df2, pcs, on='IID', how='left')
print(final_df3.shape)
final_df3.head()

In [ ]:
final_df3.drop(['#FID', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10'], axis='columns', inplace=True)

In [ ]:
print(final_df3.shape)
final_df3.head()

In [ ]:
final_df3.groupby(['PHENO'])['SEX'].value_counts()

In [ ]:
## Save your covariate file
final_df3.to_csv(f'{WORK_DIR}/{ancestry}_covariate_file.txt', sep = '\t', index=False)

In [23]:
## Save your covariate file
final_df3=pd.read_csv(f'{WORK_DIR}/{ancestry}_covariate_file.txt', sep = '\t')

In [24]:
final_df3.isna().sum()

FID       0
IID       0
MATID     0
FATID     0
SEX       0
PHENO     0
AGE       0
PC1      24
PC2      24
PC3      24
PC4      24
PC5      24
dtype: int64

In [33]:
final_df3.dropna(inplace=True)

In [34]:
final_df3.isna().sum()

FID      0
IID      0
MATID    0
FATID    0
SEX      0
PHENO    0
AGE      0
PC1      0
PC2      0
PC3      0
PC4      0
PC5      0
dtype: int64

In [ ]:
!head {WORK_DIR}/{ancestry}_covariate_file.txt

In [35]:
## Save your covariate file
final_df3.to_csv(f'{WORK_DIR}/{ancestry}_covariate_file.txt', sep = '\t', index=False)

In [ ]:
## check to make sure file was created and saved
! ls -lh {WORK_DIR}

In [ ]:
! ls -lh {AMP_DATA_DIR}

In [ ]:
## extract ancestry

! /home/jupyter/tools/plink2 \
--pfile {AMP_DATA_DIR}/chr3 \
--chr 3 \
--from-bp 132417502  \
--to-bp 132539032 \
--keep {WORK_DIR}/{ancestry}.samplestoKeep \
--pheno {WORK_DIR}/{ancestry}_covariate_file.txt \
--pheno-name PHENO \
--make-pgen \
--out {WORK_DIR}/chr3_{ancestry}

In [ ]:
# Save to workspace bucket (move from VM to workspace bucket) 
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR}/{ancestry}_covariate_file.txt {WORKSPACE_BUCKET}/2024_BURDEN_AMP_{ancestry}/{ancestry}_covariate_file.txt')
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR}/{ancestry}.samplestoKeep {WORKSPACE_BUCKET}/2024_BURDEN_AMP_{ancestry}/{ancestry}.samplestoKeep')

In [ ]:
## Check workspace bucket
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} ls {WORKSPACE_BUCKET}/2024_BURDEN_AMP_{ancestry}/')

# Annotation using ANNOVAR

* *DNAJC13* from NCBI gene
* hg38 (chr3:132417502-132539032)

In [5]:
## extract region using plink

! /home/jupyter/tools/plink2 --pfile {WORK_DIR}/chr3_{ancestry} \
--chr 3 \
--from-bp 132417502  \
--to-bp 132539032 \
--recode vcf id-paste=iid \
--mac 2 \
--out {WORK_DIR}/{ancestry}_DNAJC13

PLINK v2.00a6LM AVX2 AMD (4 Jul 2024)          www.cog-genomics.org/plink/2.0/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to 2024_BURDEN_AMP_AJ_GLM/AJ_DNAJC13.log.
Options in effect:
  --chr 3
  --export vcf id-paste=iid
  --from-bp 132417502
  --mac 2
  --out 2024_BURDEN_AMP_AJ_GLM/AJ_DNAJC13
  --pfile 2024_BURDEN_AMP_AJ_GLM/chr3_AJ
  --to-bp 132539032

Start time: Sun Aug 18 04:43:25 2024
7450 MiB RAM detected, ~5685 available; reserving 3725 MiB for main workspace.
Using up to 2 compute threads.
454 samples (0 females, 0 males, 454 ambiguous; 454 founders) loaded from
2024_BURDEN_AMP_AJ_GLM/chr3_AJ.psam.
6005 variants loaded from 2024_BURDEN_AMP_AJ_GLM/chr3_AJ.pvar.
Note: No phenotype data present.
Calculating allele frequencies... done.
5369 variants removed due to allele frequency threshold(s)
(--maf/--max-maf/--mac/--max-mac).
636 variants remaining after main filters.
--export vcf to 2024_BURDEN_AMP_AJ_GLM/AJ_DNAJC13.vcf ... 1010111112

In [6]:
### Bgzip and Tabix (zip and index the file)
! bgzip -f {WORK_DIR}/{ancestry}_DNAJC13.vcf
! tabix -f -p vcf {WORK_DIR}/{ancestry}_DNAJC13.vcf.gz

In [7]:
## annotate using ANNOVAR
! perl /home/jupyter/tools/annovar/table_annovar.pl {WORK_DIR}/{ancestry}_DNAJC13.vcf.gz /home/jupyter/tools/annovar/humandb/ -buildver hg38 \
-out {WORK_DIR}/{ancestry}_DNAJC13.annovar \
-remove -protocol refGene,clinvar_20140902 \
-operation g,f \
--nopolish \
-nastring . \
-vcfinput


NOTICE: Running with system command <convert2annovar.pl  -includeinfo -allsample -withfreq -format vcf4 2024_BURDEN_AMP_AJ_GLM/AJ_DNAJC13.vcf.gz > 2024_BURDEN_AMP_AJ_GLM/AJ_DNAJC13.annovar.avinput>
NOTICE: Finished reading 697 lines from VCF file
NOTICE: A total of 636 locus in VCF file passed QC threshold, representing 526 SNPs (341 transitions and 185 transversions) and 289 indels/substitutions
NOTICE: Finished writing allele frequencies based on 238804 SNP genotypes (154814 transitions and 83990 transversions) and 131206 indels/substitutions for 454 samples

NOTICE: Running with system command </home/jupyter/tools/annovar/table_annovar.pl 2024_BURDEN_AMP_AJ_GLM/AJ_DNAJC13.annovar.avinput /home/jupyter/tools/annovar/humandb/ -buildver hg38 -outfile 2024_BURDEN_AMP_AJ_GLM/AJ_DNAJC13.annovar -remove -protocol refGene,clinvar_20140902 -operation g,f --nopolish -nastring . -otherinfo>
-----------------------------------------------------------------
NOTICE: Processing operation=g protoc

In [8]:
# Read in ANNOVAR multianno file
gene = pd.read_csv(f'{WORK_DIR}/{ancestry}_DNAJC13.annovar.hg38_multianno.txt', sep = '\t')
display(gene)

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo457,Otherinfo458,Otherinfo459,Otherinfo460,Otherinfo461,Otherinfo462,Otherinfo463,Otherinfo464,Otherinfo465,Otherinfo466
0,3,132417864,132417864,G,C,intronic,DNAJC13,.,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
1,3,132418296,132418296,C,T,intronic,DNAJC13,.,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
2,3,132418415,132418415,A,G,intronic,DNAJC13,.,.,.,...,0/0,1/1,0/0,0/0,0/0,0/0,0/0,1/1,0/0,0/0
3,3,132418488,132418488,G,T,intronic,DNAJC13,.,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
4,3,132418506,132418506,T,G,intronic,DNAJC13,.,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873,3,132538575,132538575,A,G,UTR3,DNAJC13,NM_001329126:c.*293A>G;NM_015268:c.*293A>G,.,.,...,0/0,1/1,0/0,0/0,0/0,0/0,0/0,1/1,0/0,0/0
874,3,132538679,132538679,A,T,UTR3,DNAJC13,NM_001329126:c.*397A>T;NM_015268:c.*397A>T,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
875,3,132538787,132538787,T,G,UTR3,DNAJC13,NM_001329126:c.*505T>G;NM_015268:c.*505T>G,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
876,3,132538890,132538890,G,A,UTR3,DNAJC13,NM_001329126:c.*608G>A;NM_015268:c.*608G>A,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0


In [9]:
## Filter intronic
intronic = gene[(gene['Func.refGene'] == 'intronic')]

In [10]:
## Filter UTR3
utr3 = gene[(gene['Func.refGene'] == 'UTR3')]

In [11]:
## Filter UTR5
utr5 = gene[(gene['Func.refGene'] == 'UTR5')]

In [12]:
## Filter exonic and synonymous variants
coding_synonymous = gene[(gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] == 'synonymous SNV')]

In [13]:
# Filter exonic and non-synonymous variants
coding_nonsynonymous = gene[(gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] == 'nonsynonymous SNV')]
coding_nonsynonymous

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo457,Otherinfo458,Otherinfo459,Otherinfo460,Otherinfo461,Otherinfo462,Otherinfo463,Otherinfo464,Otherinfo465,Otherinfo466
121,3,132434618,132434618,A,G,exonic,DNAJC13,.,nonsynonymous SNV,"DNAJC13:NM_001329126:exon2:c.A68G:p.K23R,DNAJC...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
197,3,132447465,132447465,G,A,exonic,DNAJC13,.,nonsynonymous SNV,"DNAJC13:NM_001329126:exon4:c.G289A:p.A97T,DNAJ...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
444,3,132475009,132475009,C,A,exonic,DNAJC13,.,nonsynonymous SNV,"DNAJC13:NM_015268:exon22:c.C2369A:p.S790Y,DNAJ...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
595,3,132499231,132499231,C,T,exonic,DNAJC13,.,nonsynonymous SNV,"DNAJC13:NM_015268:exon37:c.C4262T:p.A1421V,DNA...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
600,3,132499777,132499777,G,A,exonic,DNAJC13,.,nonsynonymous SNV,"DNAJC13:NM_015268:exon38:c.G4385A:p.R1462H,DNA...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
601,3,132499779,132499779,G,T,exonic,DNAJC13,.,nonsynonymous SNV,"DNAJC13:NM_015268:exon38:c.G4387T:p.A1463S,DNA...",...,1/1,1/1,0/1,0/0,0/1,0/0,0/1,1/1,0/1,0/0
614,3,132502295,132502295,C,T,exonic,DNAJC13,.,nonsynonymous SNV,"DNAJC13:NM_015268:exon40:c.C4543T:p.P1515S,DNA...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/1,0/0,0/0,0/0
615,3,132502299,132502299,G,A,exonic,DNAJC13,.,nonsynonymous SNV,"DNAJC13:NM_015268:exon40:c.G4547A:p.R1516H,DNA...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
684,3,132507256,132507256,A,G,exonic,DNAJC13,.,nonsynonymous SNV,"DNAJC13:NM_015268:exon43:c.A5018G:p.Y1673C,DNA...",...,0/0,0/1,0/0,0/0,0/0,0/0,0/0,1/1,0/0,0/0
790,3,132528316,132528316,T,G,exonic,DNAJC13,.,nonsynonymous SNV,"DNAJC13:NM_015268:exon54:c.T6509G:p.L2170W,DNA...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/1,0/0,0/0


In [14]:
totalVariants = len(gene.axes[0])
totalIntronic = len(intronic.axes[0])
totalUTR3 = len(utr3.axes[0])
totalUTR5 = len(utr5.axes[0])
totalExonicSyn = len(coding_synonymous.axes[0])
totalExonicNonSyn = len(coding_nonsynonymous.axes[0])

In [15]:
print("Total Variants: ", totalVariants)
print("Intronic:", totalIntronic)
print("UTR3:", totalUTR3)
print("UTR5:", totalUTR5)
print("Exonic Syn:", totalExonicSyn)
print("Exonic NonSyn:", totalExonicNonSyn)

Total Variants:  878
Intronic: 856
UTR3: 6
UTR5: 0
Exonic Syn: 6
Exonic NonSyn: 10


In [16]:
# Save in PLINK format 
variants_toKeep = coding_nonsynonymous[['Chr', 'Start', 'End', 'Gene.refGene']].copy()
variants_toKeep.to_csv(f'{WORK_DIR}/{ancestry}_DNAJC13.all_coding_nonsyn.variantstoKeep', sep="\t", index=False, header=False)
variants_toKeep

,Chr,Start,End,Gene.refGene
121,3,132434618,132434618,DNAJC13
197,3,132447465,132447465,DNAJC13
444,3,132475009,132475009,DNAJC13
595,3,132499231,132499231,DNAJC13
600,3,132499777,132499777,DNAJC13
601,3,132499779,132499779,DNAJC13
614,3,132502295,132502295,DNAJC13
615,3,132502299,132502299,DNAJC13
684,3,132507256,132507256,DNAJC13
790,3,132528316,132528316,DNAJC13


In [17]:
## check to make sure file was created and saved
! ls {WORK_DIR}

AJ_covariate_file.txt
AJ_DNAJC13.all_coding_nonsyn.variantstoKeep
AJ_DNAJC13.annovar.avinput
AJ_DNAJC13.annovar.hg38_multianno.txt
AJ_DNAJC13.annovar.hg38_multianno.vcf
AJ_DNAJC13.log
AJ_DNAJC13.vcf.gz
AJ_DNAJC13.vcf.gz.tbi
AJ.samplestoKeep
amp_pd_participant_wgs_duplicates.csv
AMPPD_v25_GenoTools_Predictions.txt
AMPPD_v3_COV_wPHENOS_wGENOTOOLS.csv
chr3_AJ.log
chr3_AJ.pgen
chr3_AJ.psam
chr3_AJ.pvar
Demographics.csv
Enrollment.csv
PCA.FILTERED.AMP_PD_AJ.PD.eigenvec
plink2
plink2_linux_amd_avx2_20240806.zip
plink2_linux_avx2_20240625.zip
plink2_linux_avx2_20240806.zip
refFlat_HG38_all_chr.txt
toRemove_1stand2ndDegree_Relateds_AJ.txt


In [ ]:
# Save to workspace bucket (move from VM to workspace bucket) (not copying right now)
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR}/{ancestry}_DNAJC13.all_coding_nonsyn.variantstoKeep {WORKSPACE_BUCKET}/2024_BURDEN_AMP_{ancestry}/{ancestry}_DNAJC13.all_coding_nonsyn.variantstoKeep ')

In [ ]:
## Check workspace bucket
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} ls {WORKSPACE_BUCKET}/2024_BURDEN_AMP_{ancestry}/')

# Burden Analyses using RVTests

In [20]:
# Convert the files from Plink 2.0 to Plink 1.9 format 

! /home/jupyter/tools/plink2 --pfile {WORK_DIR}/chr3_{ancestry} \
--make-bed \
--max-alleles 2 \
--out {WORK_DIR}/chr3_{ancestry}

PLINK v2.00a6LM AVX2 AMD (4 Jul 2024)          www.cog-genomics.org/plink/2.0/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to 2024_BURDEN_AMP_AJ_GLM/chr3_AJ.log.
Options in effect:
  --make-bed
  --max-alleles 2
  --out 2024_BURDEN_AMP_AJ_GLM/chr3_AJ
  --pfile 2024_BURDEN_AMP_AJ_GLM/chr3_AJ

Start time: Sun Aug 18 04:46:15 2024
7450 MiB RAM detected, ~6043 available; reserving 3725 MiB for main workspace.
Using up to 2 compute threads.
454 samples (0 females, 0 males, 454 ambiguous; 454 founders) loaded from
2024_BURDEN_AMP_AJ_GLM/chr3_AJ.psam.
5615 out of 6005 variants loaded from 2024_BURDEN_AMP_AJ_GLM/chr3_AJ.pvar.
Note: No phenotype data present.
5615 variants remaining after main filters.
Writing 2024_BURDEN_AMP_AJ_GLM/chr3_AJ.fam ... done.
Writing 2024_BURDEN_AMP_AJ_GLM/chr3_AJ.bim ... done.
Writing 2024_BURDEN_AMP_AJ_GLM/chr3_AJ.bed ... done.
End time: Sun Aug 18 04:46:15 2024


In [21]:
## extract variants
    # later do based on class and frequency

! /home/jupyter/tools/plink \
--bfile {WORK_DIR}/chr3_{ancestry} \
--keep {WORK_DIR}/{ancestry}.samplestoKeep \
--extract range {WORK_DIR}/{ancestry}_DNAJC13.all_coding_nonsyn.variantstoKeep \
--recode vcf-iid \
--out {WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to 2024_BURDEN_AMP_AJ_GLM/AJ_DNAJC13.coding_nonsyn.log.
Options in effect:
  --bfile 2024_BURDEN_AMP_AJ_GLM/chr3_AJ
  --extract range 2024_BURDEN_AMP_AJ_GLM/AJ_DNAJC13.all_coding_nonsyn.variantstoKeep
  --keep 2024_BURDEN_AMP_AJ_GLM/AJ.samplestoKeep
  --out 2024_BURDEN_AMP_AJ_GLM/AJ_DNAJC13.coding_nonsyn
  --recode vcf-iid

7450 MB RAM detected; reserving 3725 MB for main workspace.
5615 variants loaded from .bim file.
454 people (0 males, 0 females, 454 ambiguous) loaded from .fam.
Ambiguous sex IDs written to
2024_BURDEN_AMP_AJ_GLM/AJ_DNAJC13.coding_nonsyn.nosex .
--extract range: 5605 variants excluded.
--extract range: 10 variants remaining.
--keep: 454 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 454 founders and 0 nonfounders present.
Calculating allele freque

In [22]:
### Bgzip and Tabix (zip and index the file)
! bgzip -f {WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn.vcf
! tabix -f -p vcf {WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn.vcf.gz

In [25]:
## RVtests with covariates 
! /home/jupyter/tools/rvtests/executable/rvtest --noweb --hide-covar \
--out {WORK_DIR}/{ancestry}_DNAJC13.burden.coding_nonsyn \
--kernel skat,skato \
--inVcf {WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn.vcf.gz \
--pheno {WORK_DIR}/{ancestry}_covariate_file.txt \
--pheno-name PHENO \
--gene DNAJC13 \
--geneFile {WORK_DIR}/refFlat_HG38_all_chr.txt #\
--covar {WORK_DIR}/{ancestry}_covariate_file.txt \
--covar-name SEX AGE PC1 PC2 PC3 PC4 PC5
#[WARN]Covariate [ PC5 ] has strong correlation [ r^2 = 1 ] with the response!


# --out : Name of output 
# --burden cmc --kernel skato: tests to run 
# --inVcf : VCF file 
# --gene: gene name (if only looking at one or a few)
# --geneFile refFlat.txt
# --pheno :  covar file
# --mpheno : # column that has phenotype information
# --pheno-name : column name with phenotype in file
# --covar : covar file
# --freqUpper : optional, MAF cut-off
# --covar-name : covariates, listed by column name, separated by commas (no spaces between commas)
## 0=controls; 1=cases

Thank you for using rvtests (version: 20190205, git: c86e589efef15382603300dc7f4c3394c82d69b8)
  For documentations, refer to http://zhanxw.github.io/rvtests/
  For questions and comments, plase send to Xiaowei Zhan <zhanxw@umich.edu>
  For bugs and feature requests, please submit at: https://github.com/zhanxw/rvtests/issues

The following parameters are available.  Ones with "[]" are in effect:

Available Options
      Basic Input/Output:
                          --inVcf [2024_BURDEN_AMP_AJ_GLM/AJ_DNAJC13.coding_nonsyn.vcf.gz]
                          --inBgen [], --inBgenSample [], --inKgg []
                          --out [2024_BURDEN_AMP_AJ_GLM/AJ_DNAJC13.burden.coding_nonsyn]
                          --outputRaw
       Specify Covariate: --covar [], --covar-name [], --sex
       Specify Phenotype: --pheno [2024_BURDEN_AMP_AJ_GLM/AJ_covariate_file.txt]
                          --inverseNormal, --useResidualAsPhenotype, --mpheno []
                          --pheno-name [PHENO]

In [26]:
## look at results 
! cat {WORK_DIR}/{ancestry}_DNAJC13.burden.coding_nonsyn.Skat.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	Q	Pvalue	NumPerm	ActualPerm	Stat	NumGreater	NumEqual	PermPvalue
DNAJC13	3:132417501-132539032,3:132417501-132539032	454	10	10	1531.19	0.958643	10000	1031	1531.19	1000	0	0.969932


In [27]:
! cat {WORK_DIR}/{ancestry}_DNAJC13.burden.coding_nonsyn.SkatO.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	Q	rho	Pvalue
DNAJC13	3:132417501-132539032,3:132417501-132539032	454	10	10	199.482	1	0.924805


In [28]:
## check to make sure file was created and saved
! ls {WORK_DIR}

AJ_covariate_file.txt
AJ_DNAJC13.all_coding_nonsyn.variantstoKeep
AJ_DNAJC13.annovar.avinput
AJ_DNAJC13.annovar.hg38_multianno.txt
AJ_DNAJC13.annovar.hg38_multianno.vcf
AJ_DNAJC13.burden.coding_nonsyn.log
AJ_DNAJC13.burden.coding_nonsyn.Skat.assoc
AJ_DNAJC13.burden.coding_nonsyn.SkatO.assoc
AJ_DNAJC13.coding_nonsyn.log
AJ_DNAJC13.coding_nonsyn.nosex
AJ_DNAJC13.coding_nonsyn.vcf.gz
AJ_DNAJC13.coding_nonsyn.vcf.gz.tbi
AJ_DNAJC13.log
AJ_DNAJC13.vcf.gz
AJ_DNAJC13.vcf.gz.tbi
AJ.samplestoKeep
amp_pd_participant_wgs_duplicates.csv
AMPPD_v25_GenoTools_Predictions.txt
AMPPD_v3_COV_wPHENOS_wGENOTOOLS.csv
chr3_AJ.bed
chr3_AJ.bim
chr3_AJ.fam
chr3_AJ.log
chr3_AJ.pgen
chr3_AJ.psam
chr3_AJ.pvar
Demographics.csv
Enrollment.csv
PCA.FILTERED.AMP_PD_AJ.PD.eigenvec
plink2
plink2_linux_amd_avx2_20240806.zip
plink2_linux_avx2_20240625.zip
plink2_linux_avx2_20240806.zip
refFlat_HG38_all_chr.txt
toRemove_1stand2ndDegree_Relateds_AJ.txt


In [ ]:
# Save to workspace bucket (move from VM to workspace bucket)
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR}/{ancestry}_DNAJC13.burden.coding_nonsyn.*.assoc {WORKSPACE_BUCKET}/2024_BURDEN_AMP_{ancestry}/')

In [ ]:
## Check workspace bucket
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} ls {WORKSPACE_BUCKET}/2024_BURDEN_AMP_{ancestry}/')

# Case/Control Frequencies

In [5]:
## Run basic association analysis just to get allele freq
! /home/jupyter/tools/plink \
--bfile {WORK_DIR}/chr3_{ancestry} \
--extract range {WORK_DIR}/{ancestry}_DNAJC13.all_coding_nonsyn.variantstoKeep \
--keep {WORK_DIR}/{ancestry}.samplestoKeep \
--pheno {WORK_DIR}/{ancestry}_covariate_file.txt \
--pheno-name PHENO \
--allow-no-sex \
--assoc \
--out {WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to 2024_BURDEN_AMP_AJ_GLM/AJ_DNAJC13.coding_nonsyn.log.
Options in effect:
  --allow-no-sex
  --assoc
  --bfile 2024_BURDEN_AMP_AJ_GLM/chr3_AJ
  --extract range 2024_BURDEN_AMP_AJ_GLM/AJ_DNAJC13.all_coding_nonsyn.variantstoKeep
  --keep 2024_BURDEN_AMP_AJ_GLM/AJ.samplestoKeep
  --out 2024_BURDEN_AMP_AJ_GLM/AJ_DNAJC13.coding_nonsyn
  --pheno 2024_BURDEN_AMP_AJ_GLM/AJ_covariate_file.txt
  --pheno-name PHENO

7450 MB RAM detected; reserving 3725 MB for main workspace.
5615 variants loaded from .bim file.
454 people (0 males, 0 females, 454 ambiguous) loaded from .fam.
Ambiguous sex IDs written to
2024_BURDEN_AMP_AJ_GLM/AJ_DNAJC13.coding_nonsyn.nosex .
430 phenotype values present after --pheno.
--extract range: 5605 variants excluded.
--extract range: 10 variants remaining.
--keep: 454 people remaining.
Using 1 thread (no mul

In [6]:
!head {WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn.assoc

 CHR           SNP         BP   A1      F_A      F_U   A2        CHISQ            P           OR 
   3   rs374320101  132434618    G 0.002688 0.002049    A      0.03715       0.8472        1.313 
   3   rs201731452  132447465    A 0.002688 0.004098    G       0.1208       0.7282        0.655 
   3   rs149121829  132475009    A        0 0.004098    C        1.528       0.2164            0 
   3    rs61748102  132499231    T  0.01075  0.01025    C     0.005235       0.9423         1.05 
   3    rs61748103  132499777    A  0.01075 0.008197    G       0.1496       0.6989        1.315 
   3     rs3762672  132499779    T   0.4409   0.5061    G        3.606      0.05757       0.7693 
   3    rs55825559  132502295    T  0.02957  0.02254    C       0.4184       0.5177        1.321 
   3   rs139620588  132502299    A 0.005376 0.004098    G      0.07447       0.7849        1.314 
   3    rs79953286  132507256    G  0.08602   0.1004    A       0.5122       0.4742       0.8432 


In [7]:
assoc = pd.read_csv(f'{WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn.assoc', delim_whitespace=True, usecols=['SNP', 'A1', 'A2', 'F_A', 'F_U'])
assoc.head()

,SNP,A1,F_A,F_U,A2
0,rs374320101,G,0.002688,0.002049,A
1,rs201731452,A,0.002688,0.004098,G
2,rs149121829,A,0.000000,0.004098,C
3,rs61748102,T,0.010750,0.010250,C
4,rs61748103,A,0.010750,0.008197,G


In [8]:
assoc.rename(columns = {'SNP': 'ID'}, inplace = True)

In [9]:
## extract variants

! /home/jupyter/tools/plink2 \
--bfile {WORK_DIR}/chr3_{ancestry} \
--extract range {WORK_DIR}/{ancestry}_DNAJC13.all_coding_nonsyn.variantstoKeep \
--keep {WORK_DIR}/{ancestry}.samplestoKeep \
--pheno {WORK_DIR}/{ancestry}_covariate_file.txt \
--pheno-name PHENO \
--covar {WORK_DIR}/{ancestry}_covariate_file.txt \
--covar-name SEX,AGE,PC1,PC2,PC3,PC4,PC5 \
--covar-variance-standardize \
--glm \
--out {WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn --ci 0.95

PLINK v2.00a6LM AVX2 AMD (4 Jul 2024)          www.cog-genomics.org/plink/2.0/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to 2024_BURDEN_AMP_AJ_GLM/AJ_DNAJC13.coding_nonsyn.log.
Options in effect:
  --bfile 2024_BURDEN_AMP_AJ_GLM/chr3_AJ
  --ci 0.95
  --covar 2024_BURDEN_AMP_AJ_GLM/AJ_covariate_file.txt
  --covar-name SEX,AGE,PC1,PC2,PC3,PC4,PC5
  --covar-variance-standardize
  --extract range 2024_BURDEN_AMP_AJ_GLM/AJ_DNAJC13.all_coding_nonsyn.variantstoKeep
  --glm
  --keep 2024_BURDEN_AMP_AJ_GLM/AJ.samplestoKeep
  --out 2024_BURDEN_AMP_AJ_GLM/AJ_DNAJC13.coding_nonsyn
  --pheno 2024_BURDEN_AMP_AJ_GLM/AJ_covariate_file.txt
  --pheno-name PHENO

Start time: Sun Aug 18 05:36:36 2024
7450 MiB RAM detected, ~6066 available; reserving 3725 MiB for main workspace.
Using up to 2 compute threads.
454 samples (0 females, 0 males, 454 ambiguous; 454 founders) loaded from
2024_BURDEN_AMP_AJ_GLM/chr3_AJ.fam.
5615 variants loaded from 2024_BURDEN_AMP_AJ_

In [10]:
! head {WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn.PHENO.glm.logistic.hybrid

#CHROM	POS	ID	REF	ALT	PROVISIONAL_REF?	A1	OMITTED	A1_FREQ	FIRTH?	TEST	OBS_CT	OR	LOG(OR)_SE	L95	U95	Z_STAT	P	ERRCODE
3	132434618	rs374320101	A	G	Y	G	A	0.00232558	N	ADD	430	1.20208	1.81173	0.0344972	41.8871	0.101588	0.919084	.
3	132434618	rs374320101	A	G	Y	G	A	0.00232558	N	SEX	430	0.68523	0.129709	0.531407	0.883579	-2.91421	0.0035659	.
3	132434618	rs374320101	A	G	Y	G	A	0.00232558	N	AGE	430	0.360817	0.147617	0.270169	0.481879	-6.90563	4.99816e-12	.
3	132434618	rs374320101	A	G	Y	G	A	0.00232558	N	PC1	430	1.76299	0.144115	1.32916	2.33841	3.93443	8.33943e-05	.
3	132434618	rs374320101	A	G	Y	G	A	0.00232558	N	PC2	430	3.25313	0.152445	2.4129	4.38594	7.73798	1.01006e-14	.
3	132434618	rs374320101	A	G	Y	G	A	0.00232558	N	PC3	430	0.833073	0.139121	0.634254	1.09422	-1.31277	0.18926	.
3	132434618	rs374320101	A	G	Y	G	A	0.00232558	N	PC4	430	0.827548	0.146617	0.620859	1.10305	-1.29104	0.19669	.
3	132434618	rs374320101	A	G	Y	G	A	0.00232558	N	PC5	430	1.17043	0.134019	0.900055	1.52203	1.17426	0.240292	.
3	132

In [11]:
assoc_glm = pd.read_csv(f'{WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn.PHENO.glm.logistic.hybrid', delim_whitespace=True)
assoc_add = assoc_glm[assoc_glm['TEST']=="ADD"]
assoc_add

,#CHROM,POS,ID,REF,ALT,PROVISIONAL_REF?,A1,OMITTED,A1_FREQ,FIRTH?,TEST,OBS_CT,OR,LOG(OR)_SE,L95,U95,Z_STAT,P,ERRCODE
0,3,132434618,rs374320101,A,G,Y,G,A,0.002326,N,ADD,430,1.202080,1.811730,0.034497,41.887100,0.101588,0.919084,.
8,3,132447465,rs201731452,G,A,Y,A,G,0.003488,N,ADD,430,0.965621,1.522280,0.048870,19.079800,-0.022982,0.981665,.
16,3,132475009,rs149121829,C,A,Y,A,C,0.002326,Y,ADD,430,2.344900,1.571910,0.107675,51.066300,0.542170,0.587701,.
24,3,132499231,rs61748102,C,T,Y,T,C,0.010465,N,ADD,430,0.473545,0.859488,0.087854,2.552460,-0.869713,0.384457,.
32,3,132499777,rs61748103,G,A,Y,A,G,0.009302,N,ADD,430,0.972473,0.998177,0.137476,6.879030,-0.027964,0.977691,.
40,3,132499779,rs3762672,G,T,Y,T,G,0.477907,N,ADD,430,0.791657,0.179982,0.556334,1.126520,-1.298050,0.194269,.
48,3,132502295,rs55825559,C,T,Y,T,C,0.025581,N,ADD,430,1.387290,0.565699,0.457764,4.204270,0.578664,0.562816,.
56,3,132502299,rs139620588,G,A,Y,A,G,0.004651,N,ADD,430,1.699510,1.067460,0.209750,13.770400,0.496826,0.619311,.
64,3,132507256,rs79953286,A,G,Y,G,A,0.094186,N,ADD,430,0.752730,0.318478,0.403228,1.405170,-0.891894,0.372450,.
72,3,132528316,rs140537885,T,G,Y,G,T,0.005814,N,ADD,430,0.067229,1.232240,0.006007,0.752396,-2.190840,0.028464,.


In [12]:
## Merge with allele freq from basic assoc analysis
assoc_add = pd.merge(assoc_add, assoc, on="ID", how="left")
assoc_add.head()

,#CHROM,POS,ID,REF,ALT,PROVISIONAL_REF?,A1_x,OMITTED,A1_FREQ,FIRTH?,...,LOG(OR)_SE,L95,U95,Z_STAT,P,ERRCODE,A1_y,F_A,F_U,A2
0,3,132434618,rs374320101,A,G,Y,G,A,0.002326,N,...,1.811730,0.034497,41.88710,0.101588,0.919084,.,G,0.002688,0.002049,A
1,3,132447465,rs201731452,G,A,Y,A,G,0.003488,N,...,1.522280,0.048870,19.07980,-0.022982,0.981665,.,A,0.002688,0.004098,G
2,3,132475009,rs149121829,C,A,Y,A,C,0.002326,Y,...,1.571910,0.107675,51.06630,0.542170,0.587701,.,A,0.000000,0.004098,C
3,3,132499231,rs61748102,C,T,Y,T,C,0.010465,N,...,0.859488,0.087854,2.55246,-0.869713,0.384457,.,T,0.010750,0.010250,C
4,3,132499777,rs61748103,G,A,Y,A,G,0.009302,N,...,0.998177,0.137476,6.87903,-0.027964,0.977691,.,A,0.010750,0.008197,G


In [13]:
assoc_add.drop(columns=['A1_y'])
assoc_add.rename(columns = {'A1_x': 'A1'}, inplace = True)

In [14]:
## repeat association study applying multiple testing corrections for the raw p-values (like Bonferroni post-hoc)
! /home/jupyter/tools/plink2 \
--bfile {WORK_DIR}/chr3_{ancestry} \
--extract range {WORK_DIR}/{ancestry}_DNAJC13.all_coding_nonsyn.variantstoKeep \
--keep {WORK_DIR}/{ancestry}.samplestoKeep \
--pheno {WORK_DIR}/{ancestry}_covariate_file.txt \
--pheno-name PHENO \
--covar {WORK_DIR}/{ancestry}_covariate_file.txt \
--covar-name SEX,AGE,PC1,PC2,PC3,PC4,PC5 \
--covar-variance-standardize \
--glm \
--out {WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn --ci 0.95 --adjust

PLINK v2.00a6LM AVX2 AMD (4 Jul 2024)          www.cog-genomics.org/plink/2.0/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to 2024_BURDEN_AMP_AJ_GLM/AJ_DNAJC13.coding_nonsyn.log.
Options in effect:
  --adjust
  --bfile 2024_BURDEN_AMP_AJ_GLM/chr3_AJ
  --ci 0.95
  --covar 2024_BURDEN_AMP_AJ_GLM/AJ_covariate_file.txt
  --covar-name SEX,AGE,PC1,PC2,PC3,PC4,PC5
  --covar-variance-standardize
  --extract range 2024_BURDEN_AMP_AJ_GLM/AJ_DNAJC13.all_coding_nonsyn.variantstoKeep
  --glm
  --keep 2024_BURDEN_AMP_AJ_GLM/AJ.samplestoKeep
  --out 2024_BURDEN_AMP_AJ_GLM/AJ_DNAJC13.coding_nonsyn
  --pheno 2024_BURDEN_AMP_AJ_GLM/AJ_covariate_file.txt
  --pheno-name PHENO

Start time: Sun Aug 18 05:36:54 2024
7450 MiB RAM detected, ~6064 available; reserving 3725 MiB for main workspace.
Using up to 2 compute threads.
454 samples (0 females, 0 males, 454 ambiguous; 454 founders) loaded from
2024_BURDEN_AMP_AJ_GLM/chr3_AJ.fam.
5615 variants loaded from 2024_BUR

In [15]:
! cat {WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn.PHENO.glm.logistic.hybrid.adjusted

#CHROM	ID	A1	UNADJ	GC	BONF	HOLM	SIDAK_SS	SIDAK_SD	FDR_BH	FDR_BY
3	rs140537885	G	0.0284636	0.0284636	0.284636	0.284636	0.250812	0.250812	0.284636	0.833689
3	rs3762672	T	0.194269	0.194269	1	1	0.88468	0.856876	0.884731	1
3	rs79953286	G	0.37245	0.37245	1	1	0.990527	0.975946	0.884731	1
3	rs61748102	T	0.384457	0.384457	1	1	0.992191	0.975946	0.884731	1
3	rs55825559	T	0.562816	0.562816	1	1	0.999745	0.993018	0.884731	1
3	rs149121829	A	0.587701	0.587701	1	1	0.999858	0.993018	0.884731	1
3	rs139620588	A	0.619311	0.619311	1	1	0.999936	0.993018	0.884731	1
3	rs374320101	G	0.919084	0.919084	1	1	1	0.99947	0.981665	1
3	rs61748103	A	0.977691	0.977691	1	1	1	0.999502	0.981665	1
3	rs201731452	A	0.981665	0.981665	1	1	1	0.999502	0.981665	1


In [16]:
assoc_adjusted = pd.read_csv(f'{WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn.PHENO.glm.logistic.hybrid.adjusted', delim_whitespace=True, usecols=['ID', 'BONF'])
assoc_adjusted

,ID,BONF
0,rs140537885,0.284636
1,rs3762672,1.000000
2,rs79953286,1.000000
3,rs61748102,1.000000
4,rs55825559,1.000000
5,rs149121829,1.000000
6,rs139620588,1.000000
7,rs374320101,1.000000
8,rs61748103,1.000000
9,rs201731452,1.000000


In [17]:
# Merge the results
assoc_add = pd.merge(assoc_add, assoc_adjusted, on="ID", how="left")
assoc_add.head()

,#CHROM,POS,ID,REF,ALT,PROVISIONAL_REF?,A1,OMITTED,A1_FREQ,FIRTH?,...,L95,U95,Z_STAT,P,ERRCODE,A1_y,F_A,F_U,A2,BONF
0,3,132434618,rs374320101,A,G,Y,G,A,0.002326,N,...,0.034497,41.88710,0.101588,0.919084,.,G,0.002688,0.002049,A,1.0
1,3,132447465,rs201731452,G,A,Y,A,G,0.003488,N,...,0.048870,19.07980,-0.022982,0.981665,.,A,0.002688,0.004098,G,1.0
2,3,132475009,rs149121829,C,A,Y,A,C,0.002326,Y,...,0.107675,51.06630,0.542170,0.587701,.,A,0.000000,0.004098,C,1.0
3,3,132499231,rs61748102,C,T,Y,T,C,0.010465,N,...,0.087854,2.55246,-0.869713,0.384457,.,T,0.010750,0.010250,C,1.0
4,3,132499777,rs61748103,G,A,Y,A,G,0.009302,N,...,0.137476,6.87903,-0.027964,0.977691,.,A,0.010750,0.008197,G,1.0


In [18]:
! /home/jupyter/tools/plink \
--bfile {WORK_DIR}/chr3_{ancestry} \
--extract range {WORK_DIR}/{ancestry}_DNAJC13.all_coding_nonsyn.variantstoKeep \
--pheno {WORK_DIR}/{ancestry}_covariate_file.txt \
--pheno-name PHENO \
--keep {WORK_DIR}/{ancestry}.samplestoKeep \
--recode A \
--out {WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to 2024_BURDEN_AMP_AJ_GLM/AJ_DNAJC13.coding_nonsyn.log.
Options in effect:
  --bfile 2024_BURDEN_AMP_AJ_GLM/chr3_AJ
  --extract range 2024_BURDEN_AMP_AJ_GLM/AJ_DNAJC13.all_coding_nonsyn.variantstoKeep
  --keep 2024_BURDEN_AMP_AJ_GLM/AJ.samplestoKeep
  --out 2024_BURDEN_AMP_AJ_GLM/AJ_DNAJC13.coding_nonsyn
  --pheno 2024_BURDEN_AMP_AJ_GLM/AJ_covariate_file.txt
  --pheno-name PHENO
  --recode A

7450 MB RAM detected; reserving 3725 MB for main workspace.
5615 variants loaded from .bim file.
454 people (0 males, 0 females, 454 ambiguous) loaded from .fam.
Ambiguous sex IDs written to
2024_BURDEN_AMP_AJ_GLM/AJ_DNAJC13.coding_nonsyn.nosex .
430 phenotype values present after --pheno.
--extract range: 5605 variants excluded.
--extract range: 10 variants remaining.
--keep: 454 people remaining.
phenotypes to be ignored, use the --

In [ ]:
recode = pd.read_csv(f'{WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn.raw', delim_whitespace=True)
recode.head()

In [20]:
# Make a list from the column names
column_names = recode.columns.tolist()

# Drop the first 6 columns to keep the variants 
variants = column_names[6:]

print(f'Number of variants in {ancestry} for DNAJC13: {len(variants)}')
variants

Number of variants in AJ for DNAJC13: 10


['rs374320101_G',
 'rs201731452_A',
 'rs149121829_A',
 'rs61748102_T',
 'rs61748103_A',
 'rs3762672_T',
 'rs55825559_T',
 'rs139620588_A',
 'rs79953286_G',
 'rs140537885_G']

In [21]:
# Pre-filter the dataset
cases_data = recode[recode['PHENOTYPE'] == 2]
controls_data = recode[recode['PHENOTYPE'] == 1]

results = []

for variant in variants:
    # For cases
    hom_cases = cases_data[cases_data[variant] == 2].shape[0]
    het_cases = cases_data[cases_data[variant] == 1].shape[0]
    total_cases = cases_data.shape[0]
    freq_cases = (hom_cases + het_cases) / total_cases

    # For controls
    hom_controls = controls_data[controls_data[variant] == 2].shape[0]
    het_controls = controls_data[controls_data[variant] == 1].shape[0]
    total_controls = controls_data.shape[0]
    freq_controls = (hom_controls + het_controls) / total_controls

    results.append({
        'Variant': variant,
        'Hom Cases': hom_cases,
        'Het Cases': het_cases,
        'Total Cases': total_cases,
        'Carrier freq in Cases': freq_cases,
        'Hom Controls': hom_controls,
        'Het Controls': het_controls,
        'Total Controls': total_controls,
        'Carrier freq in Controls': freq_controls
    })

# Return
df_results = pd.DataFrame(results)
df_results['ID'] = df_results['Variant'].apply(lambda x: x.rsplit('_', 1)[0])

df_results

,Variant,Hom Cases,Het Cases,Total Cases,Carrier freq in Cases,Hom Controls,Het Controls,Total Controls,Carrier freq in Controls,ID
0,rs374320101_G,0,1,186,0.005376,0,1,244,0.004098,rs374320101
1,rs201731452_A,0,1,186,0.005376,0,2,244,0.008197,rs201731452
2,rs149121829_A,0,0,186,0.000000,0,2,244,0.008197,rs149121829
3,rs61748102_T,0,4,186,0.021505,0,5,244,0.020492,rs61748102
4,rs61748103_A,0,4,186,0.021505,0,4,244,0.016393,rs61748103
5,rs3762672_T,35,94,186,0.693548,65,117,244,0.745902,rs3762672
6,rs55825559_T,0,11,186,0.059140,1,9,244,0.040984,rs55825559
7,rs139620588_A,0,2,186,0.010753,0,2,244,0.008197,rs139620588
8,rs79953286_G,2,28,186,0.161290,4,41,244,0.184426,rs79953286
9,rs140537885_G,0,1,186,0.005376,0,4,244,0.016393,rs140537885


In [22]:
#Merge case/control freq with the glm assoc analysis file
full_results = pd.merge(df_results, assoc_add,  on="ID", how="left")

In [24]:
clean_full_results = full_results[['Variant', 'Hom Cases', 'Het Cases', 'Total Cases',
                                   'Hom Controls','Het Controls', 'Total Controls',
                                   'ID','A1_FREQ','A1','F_A', 'F_U', 'A2','OBS_CT','L95', 'OR','U95','LOG(OR)_SE','Z_STAT','P', 'BONF']].copy()

In [25]:
# Make a list of significant SNPs, if any
sig_freq = clean_full_results[clean_full_results['P']<0.05]
sig_snps = sig_freq['ID'].tolist()
sig_snps

['rs140537885']

In [26]:
# Filter SNPs found significant on basic analysis, if any, for comparison purposes 
sig_df_results = clean_full_results[clean_full_results['ID'].isin(sig_snps)]
sig_df_results

,Variant,Hom Cases,Het Cases,Total Cases,Hom Controls,Het Controls,Total Controls,ID,A1_FREQ,A1,...,F_U,A2,OBS_CT,L95,OR,U95,LOG(OR)_SE,Z_STAT,P,BONF
9,rs140537885_G,0,1,186,0,4,244,rs140537885,0.005814,G,...,0.008197,T,430,0.006007,0.067229,0.752396,1.23224,-2.19084,0.028464,0.284636


In [30]:
sig_df_results["F_A"]

9    0.002688
Name: F_A, dtype: float64

In [27]:
# Save files to VM
clean_full_results.to_csv(f'{WORK_DIR}/{ancestry}_DNAJC13.fullVariantInformation.txt', sep="\t", index=False)
sig_df_results.to_csv(f'{WORK_DIR}/{ancestry}_DNAJC13.SignificantVariantInformation.txt' , sep="\t", index=False)

In [ ]:
# Save to workspace bucket (move from VM to workspace bucket)
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR}/{ancestry}_DNAJC13.annovar.hg38_multianno.txt {WORKSPACE_BUCKET}/2024_BURDEN_AMP_{ancestry}/{ancestry}_DNAJC13.annovar.hg38_multianno.txt')
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR}/{ancestry}_DNAJC13.fullVariantInformation.txt {WORKSPACE_BUCKET}/2024_BURDEN_AMP_{ancestry}/')
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR}/{ancestry}_DNAJC13.SignificantVariantInformation.txt {WORKSPACE_BUCKET}/2024_BURDEN_AMP_{ancestry}/')

shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn.raw {WORKSPACE_BUCKET}/2024_BURDEN_AMP_{ancestry}/')
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn.frq.cc {WORKSPACE_BUCKET}/2024_BURDEN_AMP_{ancestry}/')
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn.frq {WORKSPACE_BUCKET}/2024_BURDEN_AMP_{ancestry}/')
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn.assoc {WORKSPACE_BUCKET}/2024_BURDEN_AMP_{ancestry}/')
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR}/{ancestry}_DNAJC13.coding_nonsyn.assoc.adjusted {WORKSPACE_BUCKET}/2024_BURDEN_AMP_{ancestry}/')

In [ ]:
## Check workspace bucket
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} ls {WORKSPACE_BUCKET}/2024_BURDEN_AMP_{ancestry}/')